In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Data/Animelist.csv')

In [3]:
tv = data[data['type'] == 'TV']


In [4]:
genres = tv['genre'].str.get_dummies(sep=', ')
genre_encoded = pd.concat([tv, genres], axis=1)

In [5]:
source_encoded = pd.get_dummies(genre_encoded['source'], prefix='source').astype(int)
data_filtered = pd.concat([genre_encoded, source_encoded], axis=1)

In [6]:
data_filtered

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,source_Manga,source_Music,source_Novel,source_Original,source_Other,source_Picture book,source_Radio,source_Unknown,source_Visual novel,source_Web manga
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,0,0,0,1,0,0,0,0,0,0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14414,37662,Shinya! Tensai Bakabon,NaN,深夜! 天才バカボン,Late Night! Genius Bakabon,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,0,Not yet aired,...,1,0,0,0,0,0,0,0,0,0
14431,37300,Xiao Li Yu Li Xian Ji,The Adventures of Little Carp,小鲤鱼历险记,"小鲤鱼历险记,",https://myanimelist.cdn-dena.com/images/anime/...,TV,Other,52,Finished Airing,...,0,0,0,0,1,0,0,0,0,0
14442,37428,Chara to Otamajakushi Shima,NaN,キャラとおたまじゃくし島,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,0,Currently Airing,...,0,0,0,1,0,0,0,0,0,0
14463,37787,Space Bug,NaN,スペースバグ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,0,Not yet aired,...,0,0,0,1,0,0,0,0,0,0


In [7]:
data_filtered

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,source_Manga,source_Music,source_Novel,source_Original,source_Other,source_Picture book,source_Radio,source_Unknown,source_Visual novel,source_Web manga
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,0,0,0,1,0,0,0,0,0,0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14414,37662,Shinya! Tensai Bakabon,NaN,深夜! 天才バカボン,Late Night! Genius Bakabon,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,0,Not yet aired,...,1,0,0,0,0,0,0,0,0,0
14431,37300,Xiao Li Yu Li Xian Ji,The Adventures of Little Carp,小鲤鱼历险记,"小鲤鱼历险记,",https://myanimelist.cdn-dena.com/images/anime/...,TV,Other,52,Finished Airing,...,0,0,0,0,1,0,0,0,0,0
14442,37428,Chara to Otamajakushi Shima,NaN,キャラとおたまじゃくし島,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,0,Currently Airing,...,0,0,0,1,0,0,0,0,0,0
14463,37787,Space Bug,NaN,スペースバグ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,0,Not yet aired,...,0,0,0,1,0,0,0,0,0,0


In [8]:
data_filtered.to_csv('Data/animelist_filt.csv')